In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import scienceplots

import math
import os
import random
from functools import partial
from decimal import Decimal
import numpy as np
# from sklearnex import patch_sklearn; patch_sklearn()
import scipy.io as sio
import pysindy as ps
from tqdm import trange

from pymoo_ga import *
# NSGA2, DNSGA2, SMSEMOA, AGEMOEA2
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.moo.dnsga2 import DNSGA2
from pymoo.algorithms.moo.sms import SMSEMOA
from pymoo.algorithms.moo.age2 import AGEMOEA2
from pymoo.core.duplicate import ElementwiseDuplicateElimination
from pymoo.termination.default import DefaultMultiObjectiveTermination
from pymoo.optimize import minimize

from utils import *
from skimage.restoration import estimate_sigma
import bm3d
# from okridge.solvel0 import *
from solvel0 import solvel0, MIOSR
from best_subset import backward_refinement, brute_force_all_subsets
from UBIC import *
from kneed import KneeLocator
from bayesian_model_evidence import log_evidence

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.preprocessing import StandardScaler
from sklearn import covariance
from sklearn.metrics import mean_absolute_percentage_error

mrmr is not installed in the env you are using. This may cause an error in future if you try to use the (missing) lib.
L0BnB is not installed.


In [2]:
n_poly = 6
n_derivatives = 6
n_modules = 8

In [3]:
data_path = "../PDE-Discovery-EC/Datasets/"
print(os.listdir(data_path))
data = sio.loadmat(os.path.join(data_path, "kuramoto_sivishinky.mat"))
u_clean = (data['uu']).real; u = u_clean.copy()
x = data['x'].ravel()
t = data['tt'].ravel()
dt = t[1]-t[0]; dx = x[2]-x[1]

['KdV_sine_rep_big.mat', 'kuramoto_sivishinky.mat', 'lorenz100.npy', 'Wave_equation', 'KdV_rudy.mat', 'lorenz10.npy', 'KG_Exp.mat', 'burgers.mat']


### Add noise

In [4]:
np.random.seed(0)
noise_type = "gaussian"
noise_lv = float(50)
print("Noise level:", noise_lv)
noise = 0.01*np.abs(noise_lv)*(u.std())*np.random.randn(u.shape[0],u.shape[1])
u = u + noise

Noise level: 50.0


### Gaussian process
    - removing entries in x that show high std

In [5]:
# import gpax

# bm3d_file = f"./Denoised_data/ks_{noise_type}{int(noise_lv)}_bm3d.npy"
# load_denoised_data = True
# if load_denoised_data:
#     print("Loading denoised data...")
#     u = np.load(bm3d_file)
# else:
#     print("Denoising")
#     n_sampled_t = 10
#     xx = colvec(x)
#     u_std = np.ones((u.shape[0], n_sampled_t))
#     for i in range(n_sampled_t):
#         rng_key_train, rng_key_predict = gpax.utils.get_keys()
    
#         gp_model = gpax.ExactGP(1, kernel='RBF')
#         gp_model.fit(rng_key_train, xx, u[:, np.random.choice(len(t))], 
#                      num_warmup=5, num_samples=20, jitter=1e-6, 
#                      chain_method='parallel', print_summary=False)
    
#         posterior_mean, f_samples = gp_model.predict(rng_key_predict, xx)
#         u_std[:, i] = np.std(f_samples[:, 0, :], axis=0)
    
#     print(u_std.mean(), u_std.max())
#     est_sigma = u_std.mean() # max also works well

#     u = bm3d.bm3d(u, sigma_psd=est_sigma, 
#                   stage_arg=bm3d.BM3DStages.ALL_STAGES, 
#                   blockmatches=(False, False))

#     # np.save(bm3d_file, u)

np.random.seed(0)
fake_noise = np.random.normal(loc=0.0, scale=estimate_sigma(u), size=u.shape)
sigmas = estimate_sigma(u+fake_noise)*np.arange(0.1, 2., 0.1)
est_sigma = sigmas[np.argmin([((u-bm3d.bm3d(u+fake_noise, sigma_psd=sigma, stage_arg=bm3d.BM3DStages.ALL_STAGES, blockmatches=(False, False)))**2).mean() \
                              for sigma in sigmas])]
u = bm3d.bm3d(u, sigma_psd=est_sigma, 
                  stage_arg=bm3d.BM3DStages.ALL_STAGES, 
                  blockmatches=(False, False))

### Denoise

In [6]:
xt = np.array([x.reshape(-1, 1), t.reshape(1, -1)], dtype=object)
X, T = np.meshgrid(x, t)
XT = np.asarray([X, T]).T

In [7]:
function_library = ps.PolynomialLibrary(degree=n_poly, include_bias=False)

weak_lib = ps.WeakPDELibrary(
    function_library=function_library,
    derivative_order=n_derivatives,
    spatiotemporal_grid=XT,
    include_bias=True,
    K=10000
)

X_pre = np.array(weak_lib.fit_transform(np.expand_dims(u, -1)))
y_pre = weak_lib.convert_u_dot_integral(np.expand_dims(u, -1))
feature_names = np.array(weak_lib.get_feature_names(), dtype=object)

# R_path = "./Cache/"
# np.save(os.path.join(R_path, f"X_pre_ks_noise{int(noise_lv)}.npy"), X_pre)
# np.save(os.path.join(R_path, f"y_pre_ks_noise{int(noise_lv)}.npy"), y_pre)
# np.save(os.path.join(R_path, f"feature_names_ks.npy"), feature_names)

In [8]:
base_poly = np.array([[p, 0] for p in range(1, n_poly+1)])
base_derivative = np.array([[0, d] for d in range(1, n_derivatives+1)])
modules = [(0, 0)] if weak_lib.include_bias else []
modules += [(p, 0) for p in range(1, n_poly+1)] + \
            [(0, d) for d in range(1, n_derivatives+1)] + \
            [tuple(p+d) for d in base_derivative for p in base_poly]
assert len(modules) == len(weak_lib.get_feature_names())
base_features = dict(zip(modules, X_pre.T))
u_t = y_pre.copy()

In [9]:
# miosr_subsets = solvel0(X_pre, y_pre, miosr=True)

### Genetic algorithm with NSGA-II

In [10]:
pop_size = 500
problem = PdeDiscoveryProblem(n_poly, n_derivatives, n_modules, 
                              base_features, u_t, order_complexity=False)

In [11]:
load_pareto_front = False

if not load_pareto_front:
    termination = DefaultMultiObjectiveTermination(
        xtol=1e-8,
        cvtol=1e-6,
        ftol=1e-8,
        period=50,
        n_max_gen=100,
        n_max_evals=100000
    )

    from pymoo.algorithms.moo.sms import SMSEMOA

    # algorithm = NSGA2(
    #                 pop_size=pop_size, 
    #                 sampling=PopulationSampling(), 
    #                 crossover=GenomeCrossover(), 
    #                 mutation=GenomeMutation(), 
    #                 eliminate_duplicates=DuplicateElimination()
    #                 )

    # algorithm = DNSGA2(
    #                 pop_size=pop_size,
    #                 sampling=PopulationSampling(),
    #                 crossover=GenomeCrossover(),
    #                 mutation=GenomeMutation(),
    #                 eliminate_duplicates=DuplicateElimination()
    #                 )

    algorithm = SMSEMOA(
                    pop_size=pop_size,
                    sampling=PopulationSampling(),
                    crossover=GenomeCrossover(),
                    mutation=GenomeMutation(),
                    eliminate_duplicates=DuplicateElimination()
                    )

    res = minimize(problem, 
                   algorithm, 
                   termination=termination, 
                   verbose=True
                  )
    
    pareto_optimal_models = res.X
    # np.save(f"./Cache/pf_ks_noise{int(noise_lv)}.npy", pareto_optimal_models)

else:
    pareto_optimal_models = np.load(f"./Cache/pf_ks_noise{int(noise_lv)}.npy", allow_pickle=True)


n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |      500 |      5 |             - |             -
     2 |     1000 |      5 |  0.0691515728 |             f
     3 |     1500 |      7 |  0.0045281622 |         ideal
     4 |     2000 |      6 |  0.0316069190 |         ideal
     5 |     2500 |      7 |  0.1349463036 |         ideal
     6 |     3000 |      5 |  0.7500000000 |         nadir
     7 |     3500 |      5 |  0.0313109605 |             f
     8 |     4000 |      5 |  7.677469E-06 |             f
     9 |     4500 |      6 |  0.2000000000 |         nadir
    10 |     5000 |      7 |  0.1666666667 |         nadir
    11 |     5500 |      7 |  0.3333333333 |         nadir
    12 |     6000 |      7 |  0.000000E+00 |             f
    13 |     6500 |      9 |  0.0246919370 |             f
    14 |     7000 |      9 |  3.296001E-06 |             f
    15 |     7500 |      9 |  5.120566E-06 |             f
    16 |     8000 |      9 |  5.120566E-06 |            

In [12]:
### OPTIONAL ###
from operator import itemgetter

effective_candidates = frozenset()
for i in range(len(pareto_optimal_models)):
    effective_candidates = effective_candidates.union(pareto_optimal_models[i][0])
effective_candidates = sorted(effective_candidates)

new_pareto_optimal_models = []
for bs in backward_refinement([sorted([effective_candidates.index(_) for _ in list(pm[0])]) for pm in pareto_optimal_models], 
                              (problem.numericalize_genome(effective_candidates), y_pre)).get_best_subsets():
    bs = itemgetter(*bs)(effective_candidates)
    if type(bs[0]) is not tuple:
        bs = (bs,)
    new_pareto_optimal_models.append([frozenset(bs)])
pareto_optimal_models = np.array(new_pareto_optimal_models)
del new_pareto_optimal_models
pareto_optimal_models

array([[frozenset({(5, 1)})],
       [frozenset({(3, 1), (3, 0)})],
       [frozenset({(1, 1), (0, 2), (0, 4)})],
       [frozenset({(1, 1), (5, 3), (0, 2), (0, 4)})],
       [frozenset({(0, 4), (1, 1), (4, 2), (0, 2), (5, 3)})],
       [frozenset({(0, 4), (3, 4), (1, 1), (0, 2), (3, 6), (5, 3)})],
       [frozenset({(0, 4), (3, 4), (1, 1), (4, 2), (0, 2), (3, 6), (5, 3)})],
       [frozenset({(0, 4), (3, 4), (1, 1), (6, 4), (4, 2), (0, 2), (3, 6), (5, 3)})],
       [frozenset({(6, 0), (0, 4), (3, 4), (4, 3), (1, 1), (4, 2), (0, 2), (3, 6), (5, 3)})],
       [frozenset({(4, 4), (6, 0), (0, 4), (3, 4), (4, 3), (1, 1), (4, 2), (0, 2), (3, 6), (5, 3)})],
       [frozenset({(4, 4), (6, 0), (0, 4), (3, 4), (1, 5), (4, 3), (1, 1), (4, 2), (0, 2), (3, 6), (5, 3)})],
       [frozenset({(0, 4), (3, 4), (4, 3), (1, 1), (6, 6), (1, 4), (4, 2), (0, 2), (5, 0), (5, 6), (6, 0), (5, 3)})],
       [frozenset({(6, 2), (6, 0), (0, 4), (4, 3), (1, 1), (5, 4), (4, 2), (0, 6), (0, 2), (3, 6), (2, 2), (5, 3

In [13]:
from collections import Counter
from pymcdm import weights as obj_w
from compromise_programming import mcdm, ranks2decision

F = problem.evaluate(pareto_optimal_models)
obj_weights = obj_w.entropy_weights(F)
types = [-1, -1]

ranks, prefs = mcdm(F, obj_weights, types)
balance_point, most_common = ranks2decision(ranks)
F[balance_point], most_common, ranks, prefs

(array([0.03144628, 3.        ]),
 [(2, 4)],
 [array([14., 13.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.,
         12.]),
  array([14., 13.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.,
         12.]),
  array([14., 13.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.,
         12.]),
  array([14., 13.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.,
         12.])],
 [array([0.1276571 , 0.28611639, 0.97810333, 0.96750144, 0.95709412,
         0.94689505, 0.93689421, 0.92709623, 0.91749139, 0.908084  ,
         0.89886435, 0.88983389, 0.88099883, 0.8723429 ]),
  array([-0.66615702, -0.44671716,  0.18567592,  0.17609409,  0.16636991,
          0.15670342,  0.14696151,  0.1372472 ,  0.12746889,  0.11771181,
          0.10790148,  0.09808413,  0.08833291,  0.07852879]),
  array([0.25      , 0.42787604, 0.96079012, 0.94176393, 0.92261535,
         0.90351637, 0.88435255, 0.86521245, 0.84601733, 0.82684047,
         0.80761782, 0.78838914, 0.76921731, 0.7

### Intercept or NO Intercept? ###

In [14]:
# true_indices = [8, 10, 13]
# true_coefficients = [-1, -1, -1]
# true_ols = sm.OLS(y_pre, X_pre[:, true_indices]).fit()
# estimated_coefficients = true_ols.params
# print(estimated_coefficients, mean_absolute_percentage_error(true_coefficients, estimated_coefficients))
# true_ols.bic, sm.OLS(y_pre, X_pre[:, [0] + true_indices]).fit().bic